## **EXERCÍCIO 08 - LangChain Intro**

## **CESAR SCHOOL**
* Pós-graduação em Engenharia e Análise de Dados - 2023.2
* **Disciplina: Tópicos Complementares**
* Professor: **Silvan Ferreira**
* Aluno: **Allan Bispo** - apsb@cesar.school

In [1]:
import getpass
import os

os.environ["OPENAI_API_KEY"] = getpass.getpass()

··········


In [2]:
# INSTALAÇÕES

!pip install langchain_openai
!pip install langchain_core

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 1.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.0/52.0 kB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 395.9/395.9 kB 8.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 365.7/365.7 kB 18.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 38.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.9/318.9 kB 15.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 150.7/150.7 kB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.9/141.9 kB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 3.3 MB/s eta 0:00:00
  Attempting uninstall: tenacity
    Found existing installation: tenacity 9.0.0
    Uninstalling tenacity-9.0.0:
      Successfu

## Modelos

In [3]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-4o-mini")

In [5]:
response = llm.invoke("Olá")

print(response.content)

Olá! Como posso ajudar você hoje?


## Prompts

### Templates Simples

In [8]:
from langchain_core.prompts import ChatPromptTemplate

prompt_template = ChatPromptTemplate.from_template("Traduza o seguinte texto para português: {texto}")

prompt = prompt_template.invoke({"texto": "Artificial Intelligence is the future!"})
print(prompt)

messages=[HumanMessage(content='Traduza o seguinte texto para português: Artificial Intelligence is the future!')]


In [9]:
response = llm.invoke(prompt)

print(response.content)

Inteligência Artificial é o futuro!


### Templates de Mensagens

In [11]:
from langchain_core.messages import SystemMessage, HumanMessage

messages = [
    SystemMessage(content="Você é um tradutor de inglês para português. Traduza as mensagens que forem enviadas."),
    HumanMessage(content="Hello, how are you?"),
]

response = llm.invoke(messages)

print(response.content)

Olá, como você está?


In [12]:
prompt_template = ChatPromptTemplate.from_messages(
    [
        ("system", "Você é um tradutor de {lingua_origem} para {lingua_destino}. Traduza as mensagens que forem enviadas."),
        ("user", "{texto}")
    ]
)

In [19]:
prompt = prompt_template.invoke({
    "lingua_origem": "espanhol",
    "lingua_destino": "português",
    "texto": "Hoy és lunes mientras ayer fue domingo."
})

print(prompt)

messages=[SystemMessage(content='Você é um tradutor de espanhol para português. Traduza as mensagens que forem enviadas.'), HumanMessage(content='Hoy és lunes mientras ayer fue domingo.')]


In [20]:
response = llm.invoke(prompt)

print(response.content)

Hoje é segunda-feira, enquanto ontem foi domingo.


### Parsers

In [21]:
from langchain_core.output_parsers import StrOutputParser

parser = StrOutputParser()

In [22]:
output = parser.invoke(response)

print(output)

Hoje é segunda-feira, enquanto ontem foi domingo.


## Encadeamento

In [23]:
chain = prompt_template | llm | parser

In [24]:
response = chain.invoke({
    "lingua_origem": "português",
    "lingua_destino": "espanhol",
    "texto": "As praias de Recife têm tubarões!"
})

print(response)

¡Las playas de Recife tienen tiburones!


In [25]:
def translate(texto, lingua_origem, lingua_destino):
    response = chain.invoke({
        "lingua_origem": lingua_origem,
        "lingua_destino": lingua_destino,
        "texto": texto
    })
    return response

In [27]:
output = translate("The beaches of Recife have sharks!", "inglês", "francês")

print(output)

Les plages de Recife ont des requins !


## Exercícios

### Exercício 1
Crie uma `chain` que a partir de um tópico informado pelo usuário, crie uma piada.

In [28]:
prompt_template = ChatPromptTemplate.from_template("Crie uma piada com o tema: {texto}")

chain = prompt_template | llm | parser

def anedota(tema):
    prompt = prompt_template.invoke({"texto": tema})
    response = chain.invoke(prompt)

    return response

In [33]:
output = anedota("cunhado")

print(output)

Por que o cunhado levou um mapa para a festa?

Porque ele sempre se perde nas conversas!


### Exercício 2
Crie uma `chain` que classifique o sentimento de um texto de entrada em positivo, neutro ou negativo.

In [38]:
prompt_template = ChatPromptTemplate.from_template("Você é um psicólogo que trabalha detectando o sentimento contido em {texto}, classificando-os em positivo, negativo ou neutro, neutro é aquele que nem é positivo e nem é negativo.")

chain = prompt_template | llm | parser

def sentimento(texto):
    prompt = prompt_template.invoke({"texto": texto})
    response = chain.invoke(prompt)

    return response

In [39]:
lista_textos = ["O silêncio ecoava nos quatro cantos do apartamento vazio. A falta de companhia era um fardo pesado, e a saudade de um abraço apertado era quase insuportável.",
"Os dias se arrastavam, um igual ao outro. A rotina monótona e repetitiva havia anestesiado qualquer sentimento mais profundo.",
"O sol rabiscava a janela, pintando o quarto de um dourado cálido. Um novo dia se abria, cheio de promessas e oportunidades. A xícara de café fumegante na mão era o ponto final para uma explosão de felicidade.",
"Sentado à beira do lago, observei a água cristalina refletir a imagem do céu. A natureza, em sua perfeição, proporcionava um momento de tranquilidade e paz interior.",
"Ao olhar para o céu estrelado, senti uma gratidão imensa pela vida, pela família e por todas as pequenas alegrias do dia a dia. A sensação de paz era tão intensa que parecia tocar cada célula do meu corpo.",
"A injustiça era um veneno que corroía por dentro. A raiva se acumulava a cada nova mentira, e a vontade de gritar era irresistível."]


In [40]:
for i, texto in enumerate(lista_textos):
  output = sentimento(texto)
  print(f'Texto {i+1} é: {output}')
  print()

Texto 1 é: A mensagem apresenta um conteúdo que evoca sentimentos de solidão e saudade, sugerindo um estado emocional negativo. O silêncio e a falta de companhia são descritos como um "fardo pesado" e a saudade de um abraço apertado é descrita como "quase insuportável". Portanto, a classificação para essa mensagem seria **negativa**.

Texto 2 é: A mensagem apresentada expressa um sentimento negativo. O uso de expressões como "os dias se arrastavam", "rotina monótona e repetitiva" e "anestesiado qualquer sentimento mais profundo" indica uma sensação de tédio, desânimo e falta de entusiasmo. Portanto, a classificação é negativa.

Texto 3 é: A mensagem apresentada contém uma descrição de um momento positivo, evocando sentimentos de felicidade e esperança. Palavras e expressões como "dourado cálido", "novo dia", "promessas e oportunidades" e "explosão de felicidade" transmitem um tom otimista e alegre. Portanto, a classificação do sentimento contido na mensagem é **positivo**.

Texto 4 é: 

### Exercício 3
Crie uma `chain` que gere o código de uma função Python de acordo com a descrição do usuário.

In [44]:
prompt_template = ChatPromptTemplate.from_template("Gere um código de uma função da linguagem Python a partir da {descricao} do usuário.")

chain = prompt_template | llm | parser

def funcao(desc):
    prompt = prompt_template.invoke({"descricao": desc})
    response = chain.invoke(prompt)

    return response

In [42]:
problema = "Calcular o perímetro de um círculo de raio R"

In [45]:
output = funcao(problema)

print(output)

Claro! Aqui está uma função em Python que calcula o perímetro (ou circunferência) de um círculo dado o raio \( R \):

```python
import math

def calcular_perimetro_circulo(raio):
    """
    Calcula o perímetro de um círculo dado seu raio.

    Parâmetros:
    raio (float): O raio do círculo.

    Retorna:
    float: O perímetro do círculo.
    """
    if raio < 0:
        raise ValueError("O raio não pode ser negativo.")
    
    perimetro = 2 * math.pi * raio
    return perimetro

# Exemplo de uso
raio_usuario = float(input("Digite o raio do círculo: "))
perimetro = calcular_perimetro_circulo(raio_usuario)
print(f"O perímetro do círculo de raio {raio_usuario} é: {perimetro:.2f}")
```

### Explicação do código:
1. **Importação do módulo `math`**: Usamos `math.pi` para obter o valor de \( \pi \).
2. **Definição da função `calcular_perimetro_circulo`**: Essa função aceita um parâmetro `raio` e calcula o perímetro usando a fórmula \( 2 \pi R \).
3. **Verificação de valores negativos**: A

### Exercício 4
Crie uma `chain` que explique de forma simplificada um tópico geral fornecido pelo usuário e, em seguida, traduza a explicação para inglês. Utilize dois templates encadeados.

In [75]:
prompt_template_1 = ChatPromptTemplate.from_template("Explique de forma simplificada e usando no máximo 20 palavras o {topico} fornecido pelo usuário.")
prompt_template_2 = ChatPromptTemplate.from_template("Traduza a {explicacao} para o inglês de forma direta, sem introdução ou frase adicional.")

chain_1 = prompt_template_1 | llm | parser
chain_2 = prompt_template_2 | llm | parser

def resumo(texto):
    prompt_1 = prompt_template_1.invoke({"topico": texto})
    response_1 = chain_1.invoke(prompt_1)
    print(response_1)
    print()

    explicacao = response_1

    prompt_2 = prompt_template_2.invoke({"explicacao": explicacao})
    response_2 = chain_2.invoke(prompt_2)

    return response_2

In [77]:
entrada = "A creatina é um suplemento popular entre atletas, auxiliando na produção de energia muscular e melhorando o desempenho em atividades de alta intensidade.\nAtletas de diversas modalidades, como musculação, crossfit e esportes de força, utilizam a creatina para aumentar a força e a resistência muscular.\nA creatina é um composto natural encontrado principalmente na carne e no peixe.\nNo corpo, ela atua como um depósito de energia, fornecendo um impulso extra durante os exercícios."
print(entrada)

A creatina é um suplemento popular entre atletas, auxiliando na produção de energia muscular e melhorando o desempenho em atividades de alta intensidade.
Atletas de diversas modalidades, como musculação, crossfit e esportes de força, utilizam a creatina para aumentar a força e a resistência muscular.
A creatina é um composto natural encontrado principalmente na carne e no peixe.
No corpo, ela atua como um depósito de energia, fornecendo um impulso extra durante os exercícios.


In [78]:
output = resumo(entrada)

print(output)

A creatina é um suplemento que melhora a energia e o desempenho muscular em atividades intensas.

Creatine is a supplement that improves energy and muscle performance in intense activities.


### Exercício 5 - Desafio
Crie uma `chain` que responda perguntas sobre o CESAR School.

In [81]:
prompt_template = ChatPromptTemplate.from_template("Responda à seguinte pergunta sobre o CESAR School: {pergunta}")

chain = prompt_template | llm | parser

def responder_pergunta(pergunta):
    prompt = prompt_template.invoke({"pergunta": pergunta})
    response = chain.invoke(prompt)

    return response

In [80]:
lista_perguntas = [
    "O que é o CESAR School?",
    "Quais cursos de graduação o CESAR School oferece?",
    "Como é o processo seletivo para ingressar no CESAR School?",
    "Quais são os principais diferenciais do CESAR School em comparação com outras instituições?"
]

In [82]:
for i, pergunta in enumerate(lista_perguntas, 1):
    resposta = responder_pergunta(pergunta)
    print(f'Resposta {i}: {resposta}')
    print()

Resposta 1: O CESAR School é uma instituição de ensino que faz parte do CESAR (Centro de Estudos e Sistemas Avançados do Recife), localizada em Recife, Pernambuco. Focada na formação de profissionais nas áreas de tecnologia, design e inovação, a escola oferece cursos de graduação, pós-graduação e programas de formação continuada. O CESAR School é conhecido por sua abordagem prática e orientada a projetos, buscando preparar os alunos para os desafios do mercado de trabalho por meio de uma educação que combina teoria e prática, além de promover a interação com o ecossistema de inovação e tecnologia.

Resposta 2: O CESAR School oferece os seguintes cursos de graduação:

1. **Bacharelado em Ciência da Computação**
2. **Bacharelado em Engenharia de Software**
3. **Bacharelado em Design**
4. **Bacharelado em Análise e Desenvolvimento de Sistemas**

Esses cursos são voltados para formar profissionais capacitados nas áreas de tecnologia, design e inovação. Além disso, o CESAR School oferece um